Tried to establish baseline but it's struggling.... I decided I'm going to reduce both the training and test sets down to 5 classes only.... then I'll possibly be able to fine tun the model... we'll see what the baseline is!



I'm only using the first 5 classes to train and test on!!!**************************
Baselin accuracy = 0.0%






In [2]:
import numpy as np
import tensorflow as tf
import vgg16
from scipy.misc import imread, imresize, imsave, imshow
from imagenet_classes import class_names 
import matplotlib.pyplot as plt
import json
import pickle
import os
import re


%matplotlib inline

# Load Data

In [3]:
meta_1 = json.load(open("/home/tuna/Projects/CS601R/Project3/lab10/lib/cub_metadata.json", "r"))
meta_2 = pickle.load(open("/home/tuna/Projects/CS601R/Project3/lab10/lib/cub_metadata.pkl", "r"))

In [4]:
#  remove number prefixes from names
for inst in meta_2['class_names']:
    meta_2['class_names'][inst] = meta_2['class_names'][inst][4:].lower()
    

In [5]:
meta_2['class_names']

{1: 'black_footed_albatross',
 2: 'laysan_albatross',
 3: 'sooty_albatross',
 4: 'groove_billed_ani',
 5: 'crested_auklet',
 6: 'least_auklet',
 7: 'parakeet_auklet',
 8: 'rhinoceros_auklet',
 9: 'brewer_blackbird',
 10: 'red_winged_blackbird',
 11: 'rusty_blackbird',
 12: 'yellow_headed_blackbird',
 13: 'bobolink',
 14: 'indigo_bunting',
 15: 'lazuli_bunting',
 16: 'painted_bunting',
 17: 'cardinal',
 18: 'spotted_catbird',
 19: 'gray_catbird',
 20: 'yellow_breasted_chat',
 21: 'eastern_towhee',
 22: 'chuck_will_widow',
 23: 'brandt_cormorant',
 24: 'red_faced_cormorant',
 25: 'pelagic_cormorant',
 26: 'bronzed_cowbird',
 27: 'shiny_cowbird',
 28: 'brown_creeper',
 29: 'american_crow',
 30: 'fish_crow',
 31: 'black_billed_cuckoo',
 32: 'mangrove_cuckoo',
 33: 'yellow_billed_cuckoo',
 34: 'gray_crowned_rosy_finch',
 35: 'purple_finch',
 36: 'northern_flicker',
 37: 'acadian_flycatcher',
 38: 'great_crested_flycatcher',
 39: 'least_flycatcher',
 40: 'olive_sided_flycatcher',
 41: 'sciss

In [6]:
meta_2['class_names'][1]

'black_footed_albatross'

In [7]:
class_names

['tench, Tinca tinca',
 'goldfish, Carassius auratus',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'tiger shark, Galeocerdo cuvieri',
 'hammerhead, hammerhead shark',
 'electric ray, crampfish, numbfish, torpedo',
 'stingray',
 'cock',
 'hen',
 'ostrich, Struthio camelus',
 'brambling, Fringilla montifringilla',
 'goldfinch, Carduelis carduelis',
 'house finch, linnet, Carpodacus mexicanus',
 'junco, snowbird',
 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 'robin, American robin, Turdus migratorius',
 'bulbul',
 'jay',
 'magpie',
 'chickadee',
 'water ouzel, dipper',
 'kite',
 'bald eagle, American eagle, Haliaeetus leucocephalus',
 'vulture',
 'great grey owl, great gray owl, Strix nebulosa',
 'European fire salamander, Salamandra salamandra',
 'common newt, Triturus vulgaris',
 'eft',
 'spotted salamander, Ambystoma maculatum',
 'axolotl, mud puppy, Ambystoma mexicanum',
 'bullfrog, Rana catesbeiana',
 'tree frog, tree-f

In [8]:
#  clean up imageNet classes for comparison
standardized_class_names = []
for name in class_names:
    standardized_class_names.append("_".join(re.split("[^a-zA-Z\d:]+", name.lower())))
    


In [9]:
standardized_class_names

['tench_tinca_tinca',
 'goldfish_carassius_auratus',
 'great_white_shark_white_shark_man_eater_man_eating_shark_carcharodon_carcharias',
 'tiger_shark_galeocerdo_cuvieri',
 'hammerhead_hammerhead_shark',
 'electric_ray_crampfish_numbfish_torpedo',
 'stingray',
 'cock',
 'hen',
 'ostrich_struthio_camelus',
 'brambling_fringilla_montifringilla',
 'goldfinch_carduelis_carduelis',
 'house_finch_linnet_carpodacus_mexicanus',
 'junco_snowbird',
 'indigo_bunting_indigo_finch_indigo_bird_passerina_cyanea',
 'robin_american_robin_turdus_migratorius',
 'bulbul',
 'jay',
 'magpie',
 'chickadee',
 'water_ouzel_dipper',
 'kite',
 'bald_eagle_american_eagle_haliaeetus_leucocephalus',
 'vulture',
 'great_grey_owl_great_gray_owl_strix_nebulosa',
 'european_fire_salamander_salamandra_salamandra',
 'common_newt_triturus_vulgaris',
 'eft',
 'spotted_salamander_ambystoma_maculatum',
 'axolotl_mud_puppy_ambystoma_mexicanum',
 'bullfrog_rana_catesbeiana',
 'tree_frog_tree_frog',
 'tailed_frog_bell_toad_ribb

In [10]:
meta_2['train_test_split']['testing'][1][:3]

[1, 3, 6]

In [11]:
rootdir = '/home/tuna/Projects/CS601R/Project3/lab10/lib/images_sq224'

all_ims_original = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        all_ims_original.append( imresize(imread(os.path.join(subdir, file), mode='RGB'), (224, 224)))
    
all_ims_original = np.asarray(all_ims_original)

In [12]:
#  create training/test sets

test_data_original = []
test_classes_original = []

# for i in range(1, len(meta_2['train_test_split']['testing'])):
for i in range(1, 5):
    test_data_original.extend(all_ims_original[meta_2['train_test_split']['testing'][i]])
    test_classes_original.extend([i] * len(meta_2['train_test_split']['testing'][i])) 
    
    
    
train_data_original = []
train_classes_original = []

# for i in range(1, len(meta_2['train_test_split']['training'])):
for i in range(1, 5):
    train_data_original.extend(all_ims_original[meta_2['train_test_split']['training'][i]])
    train_classes_original.extend([i] * len(meta_2['train_test_split']['training'][i]))   

In [13]:
print(len(train_data_original), len(test_data_original))

(120, 118)


In [14]:
#  get class names
def get_class_names(passed_indexes):
    class_names = []
    for i in passed_indexes:
        class_names.append(meta_2['class_names'][i])
    
    return class_names

test_classes_original_names = get_class_names(test_classes_original)
train_classes_original_names = get_class_names(train_classes_original)

In [15]:
#  one hot encoded versions

train_classes_ohe = np.zeros((len(train_classes_original),5))
for i,c in enumerate(train_classes_original):
    train_classes_ohe[i,c] = 1.0
    
test_classes_ohe = np.zeros((len(test_classes_original),5))
for i,c in enumerate(test_classes_original):
    test_classes_ohe[i,c] = 1.0
    
    
train_classes_ohe[0]

array([ 0.,  1.,  0.,  0.,  0.])

# Model

In [14]:
sess = tf.Session()


imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
vgg = vgg16.vgg16( imgs, 'vgg16_weights.npz', sess )




#  example test im
img1 = imread('laska.png', mode='RGB')
img1 = imresize(img1, (224, 224))



probs = sess.run(vgg.probs, feed_dict={vgg.imgs: test_data_original[:]})
# preds = (np.argsort(prob)[::-1])[0]
# for p in preds:
#     print class_names[p], prob[p]
#  only use the highest scoring



# def calculate_accuracy(preds):

# standardized_class_names[preds[0]], prob[preds[0]]


(0, 'conv1_1_W', (3, 3, 3, 64))
(1, 'conv1_1_b', (64,))
(2, 'conv1_2_W', (3, 3, 64, 64))
(3, 'conv1_2_b', (64,))
(4, 'conv2_1_W', (3, 3, 64, 128))
(5, 'conv2_1_b', (128,))
(6, 'conv2_2_W', (3, 3, 128, 128))
(7, 'conv2_2_b', (128,))
(8, 'conv3_1_W', (3, 3, 128, 256))
(9, 'conv3_1_b', (256,))
(10, 'conv3_2_W', (3, 3, 256, 256))
(11, 'conv3_2_b', (256,))
(12, 'conv3_3_W', (3, 3, 256, 256))
(13, 'conv3_3_b', (256,))
(14, 'conv4_1_W', (3, 3, 256, 512))
(15, 'conv4_1_b', (512,))
(16, 'conv4_2_W', (3, 3, 512, 512))
(17, 'conv4_2_b', (512,))
(18, 'conv4_3_W', (3, 3, 512, 512))
(19, 'conv4_3_b', (512,))
(20, 'conv5_1_W', (3, 3, 512, 512))
(21, 'conv5_1_b', (512,))
(22, 'conv5_2_W', (3, 3, 512, 512))
(23, 'conv5_2_b', (512,))
(24, 'conv5_3_W', (3, 3, 512, 512))
(25, 'conv5_3_b', (512,))
(26, 'fc6_W', (25088, 4096))
(27, 'fc6_b', (4096,))
(28, 'fc7_W', (4096, 4096))
(29, 'fc7_b', (4096,))
(30, 'fc8_W', (4096, 1000))
(31, 'fc8_b', (1000,))


# Get Baseline Accuracy on first 500 test images

In [15]:
def get_accuracy_baseline(passed_probs, true_class_names):

    predicted_classes = []
    for probs in passed_probs:
        predicted_classes.append(standardized_class_names[np.argsort(probs)[-1]])


    num_correct = 0
    for pred, true in zip(predicted_classes, true_class_names):
        if true in pred:
            num_correct += 1
            
    return float(num_correct) / len(predicted_classes)

In [16]:
get_accuracy_baseline(probs, test_classes_original_names)

0.0

# Fine Tune

In [15]:
class vgg16_cub_ft:
    def __init__(self, imgs=None, weights=None, sess=None):
        if imgs is None:
            self.imgs = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
        else:
            self.imgs = imgs
        self.convlayers()
        self.fc_layers()
        self.probs = tf.nn.softmax(self.fc1)
#         if weights is not None and sess is not None:

        self.load_weights(weights, sess)


    def convlayers(self):
        self.parameters = []

        # zero-mean input
        with tf.name_scope('preprocess') as scope:
            mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
            images = self.imgs-mean

        # conv1_1
        with tf.name_scope('conv1_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv1_2
        with tf.name_scope('conv1_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool1
        self.pool1 = tf.nn.max_pool(self.conv1_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool1')

        # conv2_1
        with tf.name_scope('conv2_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv2_2
        with tf.name_scope('conv2_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool2
        self.pool2 = tf.nn.max_pool(self.conv2_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool2')

        # conv3_1
        with tf.name_scope('conv3_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_2
        with tf.name_scope('conv3_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_3
        with tf.name_scope('conv3_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool3
        self.pool3 = tf.nn.max_pool(self.conv3_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool3')

        # conv4_1
        with tf.name_scope('conv4_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_2
        with tf.name_scope('conv4_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_3
        with tf.name_scope('conv4_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool4
        self.pool4 = tf.nn.max_pool(self.conv4_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

        # conv5_1
        with tf.name_scope('conv5_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_2
        with tf.name_scope('conv5_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_3
        with tf.name_scope('conv5_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool5
        self.pool5 = tf.nn.max_pool(self.conv5_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

    """Add in single fully connected layer with random initialization"""
    def fc_layers(self):
        # fc1
        with tf.name_scope('fc1') as scope:
            shape = int(np.prod(self.pool5.get_shape()[1:]))
            self.fc1w = tf.Variable(tf.random_normal([shape, 5], dtype=tf.float32, stddev=1e-1))
            fc1b = tf.Variable(tf.constant(1.0, shape=[5], dtype=tf.float32), trainable=True)
            pool5_flat = tf.reshape(self.pool5, [-1, shape])
            fc1l = tf.nn.bias_add(tf.matmul(pool5_flat, self.fc1w), fc1b)
            self.fc1 = tf.nn.relu(fc1l)


    def load_weights(self, weight_file, sess):
        print("Loading Weights")
       
        weights = np.load(weight_file)        
        
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            if k[0:2] == "fc":  # skip loading FC layers
                continue
#             print i, k, np.shape(weights[k])
            sess.run(self.parameters[i].assign(weights[k]))
    
    
    """Define get next batch method"""
    def get_next_batch(self, data, classes, num_to_select):
        #  shuffle with labels
        data_shuffled_mat =  np.asarray(zip(data, classes))
        np.random.shuffle(data_shuffled_mat)

        data_mat_selected = np.asarray(list(data_shuffled_mat[:num_to_select,0]))
        data_classes_selected = np.asarray(list(data_shuffled_mat[:num_to_select,1]))

        return data_mat_selected.astype(float), data_classes_selected.astype(float)
    
    
            
    """Fine tune only the last fully connected layer"""
    def fine_tune(self, ft_imgs, ft_classes_ohe, ft_num_epochs = 10, learning_rate = 0.1, batch_size=20, 
                  accuracy_fn=None):        
        
        
#         import pdb
#         pdb.set_trace()
        
        
        #  add in loss and optimization on top of network to train
        
        #  define loss function (cross entropy)
        y_ = tf.placeholder(tf.float32, shape=[None, 5])
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(self.probs, y_))
        
        
        #  use adam optimizer
        optim = tf.train.AdamOptimizer(learning_rate).minimize(loss)
        
        
        
        init_op = tf.initialize_all_variables()
        sess.run(init_op)
        
        
        
        
        
        #  I'll just pass in the calculated accuracy each round here
        accuracy_measure = tf.placeholder(tf.float32, shape=())
        
        
        summary_writer = tf.train.SummaryWriter("./tf_logs/model_4", graph=sess.graph)
        acc_summary = tf.scalar_summary( 'accuracy', accuracy_measure )
        loss_summary = tf.scalar_summary( 'loss', loss )
        merged_summary_op = tf.merge_all_summaries()
        
        
        
        
        
        
        num_epochs = ft_num_epochs
        for i in range(num_epochs):
            batch_xs, batch_ys = self.get_next_batch(ft_imgs, ft_classes_ohe, batch_size)
            
            _, l = sess.run([optim, loss], feed_dict={self.imgs: batch_xs, y_: batch_ys})
        
            print("loss " + str(l))
            
            if i % 5 == 0:
                probs = sess.run(self.probs, feed_dict={self.imgs: batch_xs, y_: batch_ys})
                acc = accuracy_fn(probs, batch_ys)
                print("acc" + str(acc))
                
                
#                 acc = 0.25
                
    
                summary, = sess.run( [merged_summary_op],feed_dict={self.imgs: batch_xs, y_: batch_ys, 
                                                                   accuracy_measure: acc})
                summary_writer.add_summary(summary, i)
    
    
        summary_writer.close()

In [16]:
def get_accuracy_fine_tuned(passed_probs, passed_true_classes):

    predicted_classes = []
    for probs in passed_probs:
        predicted_classes.append(np.argsort(probs)[-1])
        
        
    
    true_classes = []
    for probs in passed_true_classes:
        true_classes.append(np.argsort(probs)[-1])


    num_correct = 0
    for pred, true in zip(predicted_classes, true_classes):
        if true == pred:
            num_correct += 1
            
    return float(num_correct) / len(predicted_classes)

In [17]:
sess = tf.Session()

imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
# vgg = vgg16_cub_ft( weights='vgg16_weights.npz', sess=sess )
vgg_ft = vgg16_cub_ft( weights='new_finetuned_weights.npz', sess=sess )



vgg_ft.fine_tune(train_data_original, train_classes_ohe, accuracy_fn=get_accuracy_fine_tuned,
                ft_num_epochs = 100, learning_rate = 0.1, batch_size=25)

Loading Weights
loss 1.58483
acc0.32
loss 1.70483
loss 1.58483
loss 1.74483
loss 1.66483
loss 1.66483
acc0.24
loss 1.70483
loss 1.58483
loss 1.54483
loss 1.78483
loss 1.62483
acc0.28
loss 1.62483
loss 1.62483
loss 1.78483
loss 1.54483
loss 1.70483
acc0.2
loss 1.74483
loss 1.54483
loss 1.50483
loss 1.58483
loss 1.70483
acc0.2
loss 1.66483
loss 1.74483
loss 1.70483
loss 1.46483
loss 1.46483
acc0.44
loss 1.62483
loss 1.70483
loss 1.74483
loss 1.62483
loss 1.74483
acc0.16
loss 1.74483
loss 1.66483
loss 1.74483
loss 1.58483
loss 1.70483
acc0.2
loss 1.66483
loss 1.70483
loss 1.74483
loss 1.50483
loss 1.58483
acc0.32
loss 1.62483
loss 1.74483
loss 1.54483
loss 1.70483
loss 1.74483
acc0.16
loss 1.58483
loss 1.66483
loss 1.66483
loss 1.66483
loss 1.66483
acc0.24
loss 1.66483
loss 1.50483
loss 1.62483
loss 1.66483
loss 1.58483
acc0.32
loss 1.66483
loss 1.66483
loss 1.74483
loss 1.66483
loss 1.58483
acc0.32
loss 1.66483
loss 1.70483
loss 1.54483
loss 1.54483
loss 1.70483
acc0.2
loss 1.66483
loss 

KeyboardInterrupt: 

In [17]:
sess = tf.Session()

imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
# vgg = vgg16_cub_ft( weights='vgg16_weights.npz', sess=sess )
vgg_ft = vgg16_cub_ft( weights='new_finetuned_weights.npz', sess=sess )



vgg_ft.fine_tune(train_data_original, train_classes_ohe, accuracy_fn=get_accuracy_fine_tuned,
                ft_num_epochs = 50, learning_rate = 0.01, batch_size=30)

Loading Weights
loss 1.6715
acc0.233333333333
loss 1.63817
loss 1.8715
loss 1.63817
loss 1.80483
loss 1.70483
acc0.2
loss 1.7715
loss 1.63817
loss 1.6715
loss 1.6715
loss 1.7715
acc0.133333333333
loss 1.63817
loss 1.50483
loss 1.63817
loss 1.6715
loss 1.70483
acc0.2
loss 1.73817
loss 1.63817
loss 1.50483
loss 1.80483
loss 1.63817
acc0.266666666667
loss 1.7715
loss 1.60483
loss 1.6715
loss 1.7715
loss 1.63817
acc0.266666666667
loss 1.63817
loss 1.70483
loss 1.70483
loss 1.63817
loss 1.7715
acc0.133333333333
loss 1.6715
loss 1.63817
loss 1.63817


KeyboardInterrupt: 

In [ ]:
sess = tf.Session()

imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
# vgg = vgg16_cub_ft( weights='vgg16_weights.npz', sess=sess )
vgg_ft = vgg16_cub_ft( weights='new_finetuned_weights.npz', sess=sess )



vgg_ft.fine_tune(train_data_original, train_classes_ohe, accuracy_fn=get_accuracy_fine_tuned,
                ft_num_epochs = 50, learning_rate = 0.001, batch_size=35)

Loading Weights
loss 1.61912
acc0.285714285714
loss 1.64769
loss 1.67626
loss 1.76198
loss 1.67626
loss 1.76198
acc0.142857142857
loss 1.56198
loss 1.64769
loss 1.61912
loss 1.67626
loss 1.67626
acc0.228571428571
loss 1.50483
loss 1.64769
loss 1.76198
loss 1.64769
loss 1.61912
acc0.285714285714
loss 1.64769
loss 1.64769
loss 1.61912
loss 1.64769
loss 1.70483
acc0.2
loss 1.64769
loss 1.61912
loss 1.64769
loss 1.64769


In [ ]:
sess = tf.Session()

imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
# vgg = vgg16_cub_ft( weights='vgg16_weights.npz', sess=sess )
vgg_ft = vgg16_cub_ft( weights='new_finetuned_weights.npz', sess=sess )



vgg_ft.fine_tune(train_data_original, train_classes_ohe, accuracy_fn=get_accuracy_fine_tuned,
                ft_num_epochs = 50, learning_rate = 0.001, batch_size=60)

Loading Weights
loss 1.6215
acc0.283333333333
loss 1.6715
loss 1.7215
loss 1.6715
loss 1.63817
loss 1.65483
acc0.25
loss 1.7215
loss 1.68817
loss 1.63817
loss 1.6715
loss 1.68817
acc0.216666666667
loss 1.6215
loss 1.65483
loss 1.6715


In [72]:
x = tf.placeholder(tf.float32, shape=(1))
# y = tf.matmul(x, x)

y = tf.mul(0.4, x)

with tf.Session() as sess:
#   print(sess.run(y))  # ERROR: will fail because x was not fed.

#   rand_array = np.random.rand(1024, 1024)
  print(sess.run(y, feed_dict={x: [0.5]}))  # Will succeed.

[ 0.2]


### Run Fine Tune Model

In [37]:
sess = tf.Session()

imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
# vgg = vgg16_cub_ft( weights='vgg16_weights.npz', sess=sess )
vgg_ft = vgg16_cub_ft( weights='new_finetuned_weights.npz', sess=sess )



vgg_ft.fine_tune(train_data_original, train_classes_ohe, accuracy_fn=get_accuracy_fine_tuned,
                ft_num_epochs = 10, learning_rate = 0.1, batch_size=20)

Loading Weights
loss 1.89006
acc0.0
loss 1.75483
acc0.15
loss 1.79006
acc0.1
loss 1.75483
acc0.15
loss 1.60483
acc0.3
loss 1.74006
acc0.2
loss 1.67529
acc0.2
loss 1.85483
acc0.05
loss 1.75483
acc0.15
loss 1.74006
acc0.15


In [16]:
import sklearn

In [21]:
sess = tf.Session()


imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
vgg = vgg16.vgg16( imgs, 'vgg16_weights.npz', sess )




#  TODO:  Feed every training image through the network and save the output probabilties
#  then take these probabiliities and feed them into a random forest network trained with the real classes.



# #  example test im
# img1 = imread('laska.png', mode='RGB')
# img1 = imresize(img1, (224, 224))



class_probs_rf_feature = sess.run(vgg.probs, feed_dict={vgg.imgs: train_data_original[:]})

(0, 'conv1_1_W', (3, 3, 3, 64))
(1, 'conv1_1_b', (64,))
(2, 'conv1_2_W', (3, 3, 64, 64))
(3, 'conv1_2_b', (64,))
(4, 'conv2_1_W', (3, 3, 64, 128))
(5, 'conv2_1_b', (128,))
(6, 'conv2_2_W', (3, 3, 128, 128))
(7, 'conv2_2_b', (128,))
(8, 'conv3_1_W', (3, 3, 128, 256))
(9, 'conv3_1_b', (256,))
(10, 'conv3_2_W', (3, 3, 256, 256))
(11, 'conv3_2_b', (256,))
(12, 'conv3_3_W', (3, 3, 256, 256))
(13, 'conv3_3_b', (256,))
(14, 'conv4_1_W', (3, 3, 256, 512))
(15, 'conv4_1_b', (512,))
(16, 'conv4_2_W', (3, 3, 512, 512))
(17, 'conv4_2_b', (512,))
(18, 'conv4_3_W', (3, 3, 512, 512))
(19, 'conv4_3_b', (512,))
(20, 'conv5_1_W', (3, 3, 512, 512))
(21, 'conv5_1_b', (512,))
(22, 'conv5_2_W', (3, 3, 512, 512))
(23, 'conv5_2_b', (512,))
(24, 'conv5_3_W', (3, 3, 512, 512))
(25, 'conv5_3_b', (512,))
(26, 'fc6_W', (25088, 4096))
(27, 'fc6_b', (4096,))
(28, 'fc7_W', (4096, 4096))
(29, 'fc7_b', (4096,))
(30, 'fc8_W', (4096, 1000))
(31, 'fc8_b', (1000,))


In [39]:
#  do the same thing for the test set
sess = tf.Session()

imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
vgg = vgg16.vgg16( imgs, 'vgg16_weights.npz', sess )


test_class_probs_rf_feature = sess.run(vgg.probs, feed_dict={vgg.imgs: test_data_original[:]})

(0, 'conv1_1_W', (3, 3, 3, 64))
(1, 'conv1_1_b', (64,))
(2, 'conv1_2_W', (3, 3, 64, 64))
(3, 'conv1_2_b', (64,))
(4, 'conv2_1_W', (3, 3, 64, 128))
(5, 'conv2_1_b', (128,))
(6, 'conv2_2_W', (3, 3, 128, 128))
(7, 'conv2_2_b', (128,))
(8, 'conv3_1_W', (3, 3, 128, 256))
(9, 'conv3_1_b', (256,))
(10, 'conv3_2_W', (3, 3, 256, 256))
(11, 'conv3_2_b', (256,))
(12, 'conv3_3_W', (3, 3, 256, 256))
(13, 'conv3_3_b', (256,))
(14, 'conv4_1_W', (3, 3, 256, 512))
(15, 'conv4_1_b', (512,))
(16, 'conv4_2_W', (3, 3, 512, 512))
(17, 'conv4_2_b', (512,))
(18, 'conv4_3_W', (3, 3, 512, 512))
(19, 'conv4_3_b', (512,))
(20, 'conv5_1_W', (3, 3, 512, 512))
(21, 'conv5_1_b', (512,))
(22, 'conv5_2_W', (3, 3, 512, 512))
(23, 'conv5_2_b', (512,))
(24, 'conv5_3_W', (3, 3, 512, 512))
(25, 'conv5_3_b', (512,))
(26, 'fc6_W', (25088, 4096))
(27, 'fc6_b', (4096,))
(28, 'fc7_W', (4096, 4096))
(29, 'fc7_b', (4096,))
(30, 'fc8_W', (4096, 1000))
(31, 'fc8_b', (1000,))


In [ ]:
class_probs_rf_feature  #  these are the feature instances...
train_classes_original  #  these are the classes

In [33]:
from sklearn.cross_validation import cross_val_score, cross_val_predict


from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier as RF


In [22]:
class_probs_rf_feature.shape

(120, 1000)

In [23]:
len(train_classes_original)

120

In [25]:
rf_clf = RandomForestClassifier()
rf_clf.fit(class_probs_rf_feature, train_classes_original)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [37]:
#  define cross validation method that returns predicted values on the
#  training set through cv

def get_cv_pred(X, y, model_class, **kwargs):
    #  construct kfolds
    kf = StratifiedKFold(y, n_folds=10, shuffle=True)
    y_pred = y.copy()
    
    #  iterate through each fold to complete predictions on the whole
    #  training set piece by piece
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        #  initialize model with passed kwargs
        model = model_class(**kwargs)
        model.fit(X_train, y_train)
        y_pred[test_index] = model.predict(X_test)
    return y_pred

def accuracy_report(X, y, model_class, model_name, **kwargs):
    y_pred = get_cv_pred(X, y, model_class, **kwargs)
    print(model_name + " Confusion Matrix")
    print(str(confusion_matrix(y, y_pred)))
    print(str(classification_report(y, y_pred)) +
          "Accuracy %f\n\n" % accuracy_score(y, y_pred))
    print("Kappa Accuracy %f\n\n" % cohen_kappa_score(y, y_pred))


def accuracy_report_y_y_pred(y, y_pred):
    print(model_name + ":\n--------------------------------------------"
          +"-----\nConfusion Matrix\n", \
      str(confusion_matrix(y, y_pred)))
    print(str(classification_report(y, y_pred)) +
          "Accuracy %f\n\n" % accuracy_score(y, y_pred))
    print("Kappa Accuracy %f\n\n" % cohen_kappa_score(y, y_pred))

In [38]:
accuracy_report(np.asarray(class_probs_rf_feature), np.asarray(train_classes_original), RF, "Random Forest")


Random Forest Confusion Matrix
[[21  5  0  4]
 [ 8 16  2  4]
 [ 0  1 27  2]
 [ 6  5  2 17]]
             precision    recall  f1-score   support

          1       0.60      0.70      0.65        30
          2       0.59      0.53      0.56        30
          3       0.87      0.90      0.89        30
          4       0.63      0.57      0.60        30

avg / total       0.67      0.68      0.67       120
Accuracy 0.675000


Kappa Accuracy 0.566667




In [40]:
test_class_probs_rf_feature.shape

(118, 1000)

In [41]:
class_probs_rf_feature.shape

(120, 1000)

In [ ]:
#  combine training and test data for cross validaiton

In [47]:
combined_probs_rf_features = np.vstack((class_probs_rf_feature, test_class_probs_rf_feature))
combined_probs_rf_features.shape

(238, 1000)

In [49]:
len(train_classes_original)

120

In [48]:
len(test_classes_original)

118

In [51]:
combined_classes = train_classes_original + test_classes_original
len(combined_classes)

238

In [52]:
accuracy_report(np.asarray(combined_probs_rf_features), np.asarray(train_classes_original), RF, "Random Forest")


Random Forest Confusion Matrix
[[23  4  0  3]
 [ 6 20  3  1]
 [ 0  1 29  0]
 [ 5  3  2 20]]
             precision    recall  f1-score   support

          1       0.68      0.77      0.72        30
          2       0.71      0.67      0.69        30
          3       0.85      0.97      0.91        30
          4       0.83      0.67      0.74        30

avg / total       0.77      0.77      0.76       120
Accuracy 0.766667


Kappa Accuracy 0.688889




# Display Current Image

In [41]:
# display the final image
foo = sess.run( opt_img )

#  use scipy's imshow?

#  need to scale between 0,1 possibly
# plt.imshow(foo.squeeze())
imshow(foo.squeeze())

In [42]:
foo = sess.run( opt_img )


imsave('generated_im.png', foo.squeeze())
